In [26]:
import sys
sys.path.append('..')
from model.models_dgl import FixedNet
import torch
import dgl
device = torch.device('cuda')
model = FixedNet(1,4,2,False,'GraphConvWL')
model.set_paramerters()
model.to(device)

FixedNet(
  (convs): ModuleList(
    (0): GraphConvWL(
      (conv_from_neigh): GraphConv(in=1, out=1, normalization=none, activation=None)
      (conv_from_self): Linear(in_features=1, out_features=1, bias=False)
    )
    (1): GraphConvWL(
      (conv_from_neigh): GraphConv(in=1, out=1, normalization=none, activation=None)
      (conv_from_self): Linear(in_features=1, out_features=1, bias=False)
    )
  )
)

In [27]:
from build_graph import BA4labelDataset,build_graph
def generate_single_sample(label, perturb_type, nodes_num = 25, m = 1, perturb_dic = {}, 
seed = None, no_attach_init_nodes=False):
    '''
    return a networkx instance
    '''
    basis_type = "ba"
    which_type = label
    if which_type == 0:
        if perturb_type == 0:
            G, role_id, plug_id = build_graph(nodes_num, basis_type, [], start = 0, m = m, seed = seed, no_attach_init_nodes=no_attach_init_nodes)
        else:
            G, role_id, plug_id = build_graph(nodes_num - perturb_type, basis_type, [[perturb_dic[perturb_type]]], start = 0, m = m, seed = seed, no_attach_init_nodes=no_attach_init_nodes)
    else:
        list_shapes = [["house"]] * (which_type - 1) + [["five_cycle"]] * (3 - which_type)
        if perturb_type != 0:
            list_shapes = list_shapes + [[perturb_dic[perturb_type]]]
        G, role_id, plug_id = build_graph(nodes_num-10-perturb_type, basis_type, list_shapes, start = 0, m = m, seed = seed, no_attach_init_nodes=no_attach_init_nodes)
    return G

In [28]:
G = generate_single_sample(2, 0, nodes_num = 25, m = 6, perturb_dic = {4:'square_diagonal'}, seed = 0, no_attach_init_nodes = True)
g = dgl.from_networkx(G)
g = g.to(device)

In [29]:
model(g, torch.ones((25,1)).to(device))

tensor([14.], device='cuda:0', grad_fn=<UnbindBackward>)


tensor([[0, 0, 1, 0]])

In [38]:
def model_forward(x, g, model):
    out = model(g, x)
    return out

In [31]:
from captum.attr import IntegratedGradients
ig = IntegratedGradients(model_forward)

In [39]:
x = torch.ones((25,1)).to(device)
x.requires_grad_()
attr = ig.attribute(x, target=2, additional_forward_args = (g,model))
attr = attr.detach().numpy()

DGLError: Expect number of features to match number of nodes (len(u)). Got 1250 and 25 instead.

In [37]:
model_forward(x, g, model)

tensor([14.], device='cuda:0', grad_fn=<UnbindBackward>)


tensor([[0, 0, 1, 0]])

In [88]:
x = torch.ones((25,1))
w = torch.tensor([2],dtype = torch.float, requires_grad=True)
b = torch.tensor([1],dtype = torch.float, requires_grad=True)
y = torch.matmul(x,w)+b
y = torch.sum(y)
if y == 75:
    z = 1
else:
    z = 0
import torch.nn.functional as F

In [89]:
z.backward()
print(w.grad)

AttributeError: 'int' object has no attribute 'backward'

In [59]:
inp = torch.eye(5, requires_grad=True)
out = (inp+1).pow(2)
out.backward(torch.ones_like(inp), retain_graph=True)


In [60]:
torch.ones_like(inp)

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])